In [ ]:
#Emotion Recognition System Using Facial Expressions and Text Analysis

In [1]:
import os
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Preprocess image function
def preprocess_image(image_path, img_size=(48, 48)):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, img_size)
    normalized = resized / 255.0
    return normalized

# Load dataset function
def load_dataset(data_dir, img_size=(48, 48)):
    data = []
    labels = []
    for label in os.listdir(data_dir):
        class_dir = os.path.join(data_dir, label)
        for img_file in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_file)
            img = preprocess_image(img_path, img_size)
            data.append(img)
            labels.append(label)
    data = np.array(data).reshape(-1, img_size[0], img_size[1], 1)
    return data, labels

# Paths to the dataset
train_data_dir = 'C:\\Users\\gopi peketi\\OneDrive\\Desktop\\project\\archive (11)\\train'
test_data_dir = 'C:\\Users\\gopi peketi\\OneDrive\\Desktop\\project\\archive (11)\\test'

# Load datasets
X_train, y_train = load_dataset(train_data_dir)
X_test, y_test = load_dataset(test_data_dir)

# Encode labels
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

# Convert labels to categorical
y_train_categorical = to_categorical(y_train_encoded)
y_test_categorical = to_categorical(y_test_encoded)

# Create the model
def create_model(input_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(len(le.classes_), activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = create_model(X_train.shape[1:])

# Train the model
model.fit(X_train, y_train_categorical, epochs=10, validation_data=(X_test, y_test_categorical))


# Save the model
model.save('facial_expression_model.h5')


C:\Users\gopi peketi\anaconda3\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 77s 83ms/step - accuracy: 0.3328 - loss: 1.6743 - val_accuracy: 0.4592 - val_loss: 1.4180
Epoch 2/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 72s 80ms/step - accuracy: 0.4709 - loss: 1.3798 - val_accuracy: 0.4778 - val_loss: 1.3418
Epoch 3/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 71s 79ms/step - accuracy: 0.5158 - loss: 1.2659 - val_accuracy: 0.4930 - val_loss: 1.3068
Epoch 4/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 72s 80ms/step - accuracy: 0.5659 - loss: 1.1485 - val_accuracy: 0.5145 - val_loss: 1.2558
Epoch 5/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 72s 80ms/step - accuracy: 0.6107 - loss: 1.0427 - val_accuracy: 0.5277 - val_loss: 1.2432
Epoch 6/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 72s 80ms/step - accuracy: 0.6581 - loss: 0.9208 - val_accuracy: 0.5242 - val_loss: 1.2924
Epoch 7/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 72s 80ms/step - accuracy: 0.7010 - loss: 0.8178 - val_accuracy: 0.5234 - val_loss: 1.3210
Epoch 8/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 84s 82ms/step - accuracy: 0.7448 - loss: 0.7132 - 

In [34]:
le.classes_ = np.array(['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral'])


In [36]:
# After training the model
np.save('classes.npy', le.classes_)


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, accuracy_score
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import re
import joblib

# Download NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Convert to lowercase
    text = text.lower()
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Tokenize and remove stopwords and lemmatize
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)

def main():
    # Load the dataset
    df = pd.read_csv('C:\\Users\\gopi peketi\\OneDrive\\Desktop\project\\archive (15)\\emotion_mapped.csv')

    # Apply preprocessing to the text column
    df['processed_text'] = df['text'].apply(preprocess_text)

    # Split the data into training and testing sets
    X = df['processed_text']
    y = df['label']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Create a pipeline with TfidfVectorizer and MultinomialNB
    model = make_pipeline(TfidfVectorizer(), MultinomialNB())

    # Train the model
    model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Print the classification report
    print(classification_report(y_test, y_pred))

    # Calculate and print the accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Model Accuracy: {accuracy * 100:.2f}%")

    # Save the trained model to a file
    joblib.dump(model, 'text_detection_model1.pkl')

def predict_emotion(text):
    # Load the model from the file
    loaded_model = joblib.load('text_detection_model1.pkl')
    
    # Preprocess the input text
    processed_text = preprocess_text(text)
    
    # Predict and return the emotion
    return loaded_model.predict([processed_text])[0]

if __name__ == "__main__":
    main()

   


[nltk_data] Downloading package stopwords to C:\Users\gopi
[nltk_data]     peketi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\gopi
[nltk_data]     peketi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


              precision    recall  f1-score   support

       Angry       0.94      0.63      0.76     11441
        Fear       0.90      0.50      0.64      9594
       Happy       0.71      0.97      0.82     28164
     Neutral       0.97      0.24      0.38      6929
         Sad       0.77      0.95      0.85     24201
    Surprise       0.98      0.08      0.15      3033

    accuracy                           0.77     83362
   macro avg       0.88      0.56      0.60     83362
weighted avg       0.81      0.77      0.74     83362

Model Accuracy: 77.16%


In [2]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder
import tkinter as tk
from tkinter import ttk, messagebox
from PIL import Image, ImageTk
import joblib
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import re

# Download NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Preprocess text function
def preprocess_text(text):
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Convert to lowercase
    text = text.lower()
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Tokenize and remove stopwords and lemmatize
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)

# Hypothetical function to predict emotion from text
def predict_emotion_from_text(text):
    # Load the model from the file
    loaded_model = joblib.load('text_detection_model1.pkl')
    
    # Preprocess the input text
    processed_text = preprocess_text(text)
    
    # Predict and return the emotion
    return loaded_model.predict([processed_text])[0]

# Load pre-trained model for facial expression recognition
facial_model = load_model('facial_expression_model.h5')

# Option 1: Load LabelEncoder classes
try:
    le = LabelEncoder()
    le.classes_ = np.load('classes.npy')
except FileNotFoundError:
    # Option 2: Manually set classes if file not found
    le = LabelEncoder()
    le.classes_ = np.array(['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral'])

# Preprocess image function
def preprocess_image(image, img_size=(48, 48)):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, img_size)
    normalized = resized / 255.0
    return normalized.reshape(1, img_size[0], img_size[1], 1)

# GUI application class
class EmotionApp(tk.Tk):
    def __init__(self):
        super().__init__()
        
        self.title("Emotion Recognition")
        self.geometry("600x400")
        
        # Configure dark mode
        self.configure(bg='#2E2E2E')
        
        style = ttk.Style()
        style.theme_use('clam')
        style.configure('TLabel', background='#2E2E2E', foreground='white')
        style.configure('TEntry', fieldbackground='#3E3E3E', foreground='white')
        style.configure('TButton', background='#3E3E3E', foreground='white')
        style.map('TButton', background=[('active', '#555555')])
        
        self.video_frame = ttk.Label(self)
        self.video_frame.pack(pady=10)
        
        self.text_input_label = ttk.Label(self, text="Enter text to predict its emotion:")
        self.text_input_label.pack(pady=5)
        
        self.text_input = ttk.Entry(self, width=50)
        self.text_input.pack(pady=5)
        
        # Define the colorful style for the button
        self.predict_button = tk.Button(self, text="Predict Emotion", command=self.predict_emotion,
                                        font=('Helvetica', 12, 'bold'), bg='#ff6347', fg='white',
                                        activebackground='#ff4500', activeforeground='white')
        self.predict_button.pack(pady=10)
        
        self.result_label = ttk.Label(self, text="", font=("Helvetica", 12))
        self.result_label.pack(pady=10)
        
        self.cap = cv2.VideoCapture(0)
        if not self.cap.isOpened():
            messagebox.showerror("Error", "Unable to access the webcam")
            self.destroy()
            return
        
        self.facial_expression_prediction = None
        self.update_video()
    
    def update_video(self):
        ret, frame = self.cap.read()
        if ret:
            faces = self.detect_faces(frame)
            for (x, y, w, h) in faces:
                face = frame[y:y+h, x:x+w]
                preprocessed_face = preprocess_image(face)
                predictions = facial_model.predict(preprocessed_face)
                max_index = np.argmax(predictions)
                self.facial_expression_prediction = le.classes_[max_index]
                cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
                cv2.putText(frame, self.facial_expression_prediction, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
            img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(img)
            imgtk = ImageTk.PhotoImage(image=img)
            self.video_frame.imgtk = imgtk
            self.video_frame.configure(image=imgtk)
        else:
            print("Error: Could not read frame from webcam.")
        self.after(10, self.update_video)
    
    def detect_faces(self, frame):
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        if face_cascade.empty():
            print("Error: Haar Cascade file not found.")
            messagebox.showerror("Error", "Failed to load Haar Cascade file")
            self.destroy()
            return []
        faces = face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=5)
        if len(faces) == 0:
            print("No faces detected.")
        return faces
    
    def predict_emotion(self):
        if self.facial_expression_prediction is not None:
            input_text = self.text_input.get()
            predicted_emotion = predict_emotion_from_text(input_text)
            result_text = f"Facial Expression: {self.facial_expression_prediction}\nText Emotion: {predicted_emotion}\n"
            if self.facial_expression_prediction.lower() == predicted_emotion.lower():
                result_text += "The facial expression matches the text emotion!"
            else:
                result_text += "The facial expression does not match the text emotion."
            self.result_label.config(text=result_text)
        else:
            messagebox.showerror("Error", "No face detected to predict facial expression.")
    
    def on_closing(self):
        self.cap.release()
        self.destroy()

# Main application entry point
if __name__ == "__main__":
    app = EmotionApp()
    app.protocol("WM_DELETE_WINDOW", app.on_closing)
    app.mainloop()


[nltk_data] Downloading package stopwords to C:\Users\gopi
[nltk_data]     peketi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\gopi
[nltk_data]     peketi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/